In [1]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\office-projects\pdf-extractor"
file_path = r"\output\TabularPdf.pdf";


In [9]:
import fitz
import tabula
import pdfplumber

In [8]:
pdf_path =r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\output\TabularPdf.pdf"

In [ ]:
tabular_data = tabula.read_pdf(pdf_path, pages='all')
print(len(tabular_data))

13


In [13]:
def extract_pdf_blocks(input_pdf_path):
    """
    Open the PDF and extract all blocks of text, images, and other content.

    Args:
        input_pdf_path (str): Path to the input PDF.

    Returns:
        list: A list of pages, where each page is a dictionary containing blocks of content.
    """
    blocks_data = []
    input_doc = fitz.open(input_pdf_path)

    with pdfplumber.open(input_pdf_path) as pdf:
        for page_number, pdf_page in enumerate(pdf.pages):
            # Extract blocks of content using PyMuPDF
            doc_page = input_doc[page_number]
            blocks = doc_page.get_text("dict")["blocks"]
            
            # Append the blocks along with table bounding boxes from pdfplumber
            blocks_data.append({
                "blocks": blocks,
                "num": page_number+1
            })

    input_doc.close()
    return blocks_data

In [25]:
doc = fitz.open(pdf_path)
new_doc = fitz.open()

for i in range(doc.page_count):
    original_page = doc.load_page(i)
    new_page = new_doc.new_page(width=original_page.rect.width, height=original_page.rect.height)

    blocks = original_page.get_text("blocks")

    for block in blocks:
        block_bbox = block[:4]
        block_text = block[4]
        
        lines = block_text.split('\n') #block has multiple lines
        y_offset = block_bbox[1] #initial y-offset for first line

        for line in lines:
            line_bbox = (block_bbox[0], y_offset, block_bbox[2], y_offset + 12)
            new_page.draw_rect(line_bbox, color=(1, 0, 0), width=1)
            new_page.insert_text((line_bbox[0], line_bbox[1] - 5), line, fontsize=8, color=(0, 0, 0))
            y_offset += 12

new_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\output\Genbboxtable.pdf"
new_doc.save(new_pdf_path)

doc.close()
new_doc.close()

print(f"New PDF with bounding boxes has been saved to: {new_pdf_path}")


New PDF with bounding boxes has been saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\output\Genbboxtable.pdf
